In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.data.lightning import LightningDataset
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch_geometric.transforms as T 

from query_strategies import *
from augmentation import *
from model import *

import wandb

from lightning_modules import *
import lightning as L
from torch_geometric.data.lightning import LightningDataset
import logging
import warnings
from util import *
from torch_geometric.data import Data, DataLoader
from trainers import *


In [2]:
warnings.filterwarnings("ignore")
logging.getLogger("lightning.pytorch").setLevel(logging.ERROR)


In [3]:
wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: banfizsombor1999. Use `wandb login --relogin` to force relogin


True

In [4]:
random.seed(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
# dataset = Planetoid(root='/tmp/Cora', name='Cora')
# generate_balanced_data_splits(dataset,10,"data_splits\\cora_splits")

In [6]:
DROPOUT= 0.3
NUM_PASSES = 10
BUDGET = 150
EPOCHS = 200
SIGNIFICANCE_ITERATIONS = 10

NOISE_PROB = 0.4
NOISE_LEVEL = 0.5

In [7]:
# Augmentations
drop_edge = DropEdge(DROPOUT)
noise_feature_all = NoiseFeature(NOISE_LEVEL, 1)
noise_feature_col = NoiseFeature(NOISE_LEVEL, NOISE_PROB, "col")
noise_feature_row = NoiseFeature(NOISE_LEVEL, NOISE_PROB, "row")
noise_latent = NoiseLatent(NOISE_LEVEL)

drop_edge_noise_all = T.Compose([drop_edge, noise_feature_all])
drop_edge_noise_col = T.Compose([drop_edge, noise_feature_col])
drop_edge_noise_row = T.Compose([drop_edge, noise_feature_row])

In [8]:
# Strategies

random_query = RandomQuery()
entropy_query = EntropyQuery()

augment_sum_entropy = AugmentGraphSumEntropyQuery(drop_edge_noise_all, NUM_PASSES,0.0)
augment_logit_change = AugmentGraphLogitChange(drop_edge_noise_all, NUM_PASSES,1.0)
augment_latent = AugmentGraphSumQueryLatent(noise_latent, NUM_PASSES)
augment_sum_entropy_with_original = AugmentGraphSumEntropyQuery(drop_edge_noise_all, NUM_PASSES, 1/NUM_PASSES)


In [9]:
data_splits = [torch.load(f"data_splits\\cora_splits\\split_{i}.pt") for i in range(10)]


In [10]:
data_splits

[Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], train_pool=[2708]),
 Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], train_pool=[2708]),
 Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], train_pool=[2708]),
 Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], train_pool=[2708]),
 Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], train_pool=[2708]),
 Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], train_pool=[2708]),
 Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], train_pool=[2708]),
 Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_m

In [11]:
dataset = data_splits[0].to(device)
num_features = dataset.x.shape[1]
model = GCN(num_features, 7).to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

trainer = Trainer(model, optimizer, loss_fn)


In [12]:
dataset.train_mask.sum(), dataset.val_mask.sum(), dataset.test_mask.sum()

(tensor(7, device='cuda:0'),
 tensor(500, device='cuda:0'),
 tensor(1000, device='cuda:0'))

In [13]:
def init_wandb(query_strategy,run):
  config={
    "learning_rate": 0.01,
    "architecture": "GCN",
    "dataset": "CORA",
    "epochs": 200,
    "strategy": str(query_strategy)
    }
  
  if hasattr(query_strategy, "augmentation_fn"):
    augmentation = query_strategy.augmentation_fn
    config["augmentations"] = []
    if isinstance(augmentation, T.Compose):
      transforms = augmentation.transforms
    else:
      transforms = [augmentation]
      
    for t in transforms:
      aug_config = {}
      aug_config["name"] = str(t)
      aug_config["hyperparameters"] = t.__hyperparameters__()
      config["augmentations"].append(aug_config)
  
  wandb.init(
    # Set the project where this run will be logged
    project="graph-active-learning",
    # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
    name=f"{query_strategy}_{run}",
    # Track hyperparameters and run metadata
    config=config)
  return config

In [ ]:
STRATEGIES = [ augment_sum_entropy_with_original,augment_logit_change]
# STRATEGIES = noises_latent
final_accs = torch.zeros((len(STRATEGIES), BUDGET, SIGNIFICANCE_ITERATIONS))
final_auc = torch.zeros((len(STRATEGIES), BUDGET, SIGNIFICANCE_ITERATIONS))

# data_splits = [generate_random_data_split(dataset,10,500) for _ in range(SIGNIFICANCE_ITERATIONS)]
data_splits = [torch.load(f"data_splits\\cora_splits\\split_{i}.pt") for i in range(10)]

for strategy_ix, query_strategy in enumerate(STRATEGIES):
    print(f"Strategy: {query_strategy}")
    init_wandb(str(query_strategy), 4)
    for b in range(1,BUDGET+1):
        
        budget_accuracies = []
        budget_aucs = []
        
        for si in range(SIGNIFICANCE_ITERATIONS):
            dataset = data_splits[si].to(device)
            
            num_features = dataset.num_features
            num_classes = dataset.y.max().item() + 1
            
            print(f"{b} - {si} - {query_strategy}")
            
            model = GCN(num_features, num_classes).to(device)
            loss_fn = torch.nn.CrossEntropyLoss()
            optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
            trainer = Trainer(model, optimizer, loss_fn)
            
            trainer.train(dataset,EPOCHS)
            acc, auc = trainer.test(dataset)
            
            budget_accuracies.append(acc)
            # budget_aucs.append(auc)
            
            query_node_idx = query_strategy(model,dataset,dataset.train_pool)
            print(f'\tQuery node: {query_node_idx}')
            
            dataset.train_mask[query_node_idx] = True
            dataset.train_pool[query_node_idx] = False

        budget_accuracies = torch.tensor(budget_accuracies)
        budget_aucs = torch.tensor(budget_aucs)
        final_accs[strategy_ix, b-1, :] = budget_accuracies
        # final_auc[strategy_ix, b-1, :] = budget_aucs
        m = budget_accuracies.mean()
        std = budget_accuracies.std()
        wandb.log({"accuracy_mean": m.item(), "step":b})
        wandb.log({"accuracy_std": std.item(), "step": b})
wandb.finish()


Strategy: AugmentGraphSumEntropyQuery([DropEdge, NoiseFeature], original_weight=0.1)


1 - 0 - AugmentGraphSumEntropyQuery([DropEdge, NoiseFeature], original_weight=0.1)
	Query node: 382
1 - 1 - AugmentGraphSumEntropyQuery([DropEdge, NoiseFeature], original_weight=0.1)
	Query node: 2221
1 - 2 - AugmentGraphSumEntropyQuery([DropEdge, NoiseFeature], original_weight=0.1)
	Query node: 2124
1 - 3 - AugmentGraphSumEntropyQuery([DropEdge, NoiseFeature], original_weight=0.1)
	Query node: 1859
1 - 4 - AugmentGraphSumEntropyQuery([DropEdge, NoiseFeature], original_weight=0.1)
	Query node: 1242
1 - 5 - AugmentGraphSumEntropyQuery([DropEdge, NoiseFeature], original_weight=0.1)
	Query node: 68
1 - 6 - AugmentGraphSumEntropyQuery([DropEdge, NoiseFeature], original_weight=0.1)
	Query node: 884
1 - 7 - AugmentGraphSumEntropyQuery([DropEdge, NoiseFeature], original_weight=0.1)
	Query node: 2153
1 - 8 - AugmentGraphSumEntropyQuery([DropEdge, NoiseFeature], original_weight=0.1)
	Query node: 2399
1 - 9 - AugmentGraphSumEntropyQuery([DropEdge, NoiseFeature], original_weight=0.1)
	Query node:

In [ ]:
wandb.finish()

accuracy_mean,▁▃▄▅▆▆▇█
accuracy_std,▅▅█▄▄▃▃▁
accuracy_mean,0.6352
accuracy_std,0.05118
